***Vorlesung 'Syntax natürlicher Sprachen', WS 2019/20***

---
# Übung 6

In [1]:
from exercises_6 import *
from questions import aufgabe
from nltk.tree import Tree
from nltk import FeatStruct
from sklearn.metrics import accuracy_score

---
## Aufgabe 1: Kongruenz 1.0
#### Betrachten Sie die folgenden Sätze. Offensichtlich besteht zwischen Artikel und Nomen einer Nominalphrase Kongruenz im Genus.
####  Nutzen Sie Merkmalstrukturen in einem Python-Skript zur Überprüfung dieser Kongruenz. Ihr Skript sollte alle untenstehenden Beispiele korrekt als grammatisch/ungrammatisch klassifizieren können.

  
1. - *Der Löffel ist aus Silber.*
   - *&ast; Der Gabel ist aus Silber.*
   - *&ast; Der Messer ist aus Silber.* <p>
    
2. - *&ast; Die Löffel ist aus Silber.* 
   - *Die Gabel ist aus Silber.*
   - *&ast; Die Messer ist aus Silber.* <p>
   
3. - *&ast; Das Löffel ist aus Silber.*
   - *&ast; Das Gabel ist aus Silber.*
   - *Das Messer ist aus Silber.*

In [2]:
test_cases = [
    (det, noun)
    for det in ["der", "die", "das"]
    for noun in ["Löffel", "Gabel", "Messer"]
]
truth = [
    True, False, False,
    False, True, False,
    False, False, True
]

In [3]:
test_cases

[('der', 'Löffel'),
 ('der', 'Gabel'),
 ('der', 'Messer'),
 ('die', 'Löffel'),
 ('die', 'Gabel'),
 ('die', 'Messer'),
 ('das', 'Löffel'),
 ('das', 'Gabel'),
 ('das', 'Messer')]

In [4]:
lexicon = {
    'der': FeatStruct(GEN='mask'),
    'die': FeatStruct(GEN='fem'),
    'das': FeatStruct(GEN='neut'),
    'Löffel': FeatStruct(GEN='mask'),
    'Gabel': FeatStruct(GEN='fem'),
    'Messer': FeatStruct(GEN='neut')
}

In [5]:
def is_correct(det, noun):
    det_fs = lexicon[det]
    noun_fs = lexicon[noun]
    return det_fs.unify(noun_fs) is not None

In [6]:
str(lexicon['der'].unify(lexicon['Löffel']))

"[ GEN = 'mask' ]"

In [7]:
lexicon['der'].unify(lexicon['Gabel']) is None

True

In [8]:
# Evaluation
pred = []
for nomPhr in test_cases:
    pred.append(is_correct(*nomPhr))

print("Accuracy: {:.2f} %".format(accuracy_score(truth, pred)*100))

Accuracy: 100.00 %


---
## Aufgabe 2: Semantische Merkmalstrukturen
#### Implementieren Sie die folgenden Merkmalstrukturen als `nltk.FeatStruct` und schreiben Sie ein Programm, das für einen gegebenen Satz die richtige Merkmalstruktur aus dem Lexikon auswählt und die Lücken für Subjekt und Objekt des Satzes füllt. Geben Sie anschließend die vollständige Merkmalstruktur auf der Standardausgabe aus.

$$
\left [ % draw the opening left bracket
\begin{array}{ll}   % begin a two column array
   CAT & V \\
   LEMMA & \text{folgen} \\
   SYN &  \left  [ \begin{array}{ll}
                        SBJ & \boxed{1}\\
                        OBJ & \boxed{2} \\     
                    \end{array} 
           \right  ]  \\
   SEM   & \left [ \begin{array}{ll}
                          AGT & \boxed{1} \\
                          PAT & \boxed{2} \\
                   \end{array}
           \right ]
\end{array} % end the outer array
\right ]    % closing right bracket
~~~~~~~~~~
\left [ % draw the opening left bracket
\begin{array}{ll}   % begin a two column array
   CAT & V \\
   LEMMA & \text{überraschen} \\
   SYN &  \left  [ \begin{array}{ll}
                        SBJ & \boxed{1}\\
                        OBJ & \boxed{2} \\     
                    \end{array} 
           \right  ]  \\
   SEM   & \left [ \begin{array}{ll}
                          SRC & \boxed{1} \\
                          EXP & \boxed{2} \\
                   \end{array}
           \right ]
\end{array} % end the outer array
\right ]    % closing right bracket
$$

#### *Anmerkung*: Sie dürfen annehmen, dass  wir es nur mit einfachen Sätzen zu tun haben, in denen es nur ein Verb und nur jeweils einen Kandidaten für Subjekt und Objekt gibt.

#### Testen Sie Ihr Programm mit den folgenden Sätzen:

In [9]:
sentences = [
   "Der Hund ist mir bis nach Hause gefolgt!",
   "Das überrascht mich doch sehr!",
   "Wer folgt wem auf Twitter?",
   "John überrascht Jane mit einer richtigen Antwort." 
]

In [10]:
lexicon = {
    'folgen': FeatStruct(
        '[CAT=V, LEMMA=folgen, SYN=[SBJ=?x, OBJ=?y], SEM=[AGT=?x, PAT=?y]]'
    ),
    'überraschen': FeatStruct(
        '[CAT=V, LEMMA=ueberraschen, SYN=[SBJ=?x, OBJ=?y], SEM=[SRC=?x, EXP=?y]]'
    )
}

In [11]:
import spacy
nlp = spacy.load('de_core_news_sm')

In [12]:
analyzed = nlp(sentences[1])
for token in analyzed:
    print(token.text, token.pos_, token.lemma_, token.dep_)

Das PRON der sb
überrascht VERB überraschen ROOT
mich PRON sich oa
doch ADV doch mo
sehr ADV sehr mo
! PUNCT ! punct


In [13]:
spacy.explain('oa')

'accusative object'

In [14]:
#  In: sentence as a string
# Out: semantic feature structure
def semantic_parse(sentence):
    # syntaktische Analyse
    analyzed = nlp(sentence)
    # Verb finden
    verb = None
    for token in analyzed:
        if token.pos_ == 'VERB' and token.lemma_ in lexicon:
            verb = token.lemma_
    # FS aus lexikon suchen
    if verb is None:
        return None
    verb_fs = lexicon[verb]
    # subjekt und objekt finden
    sbj, obj = None, None
    for token in analyzed:
        if token.dep_ == 'sb':
            sbj = token.text
        elif token.dep_ == 'da' or token.dep_ == 'oa':
            obj = token.text
    if sbj is None or obj is None:
        return None
    # geschickt unifizieren
    syn_fs = FeatStruct(SYN=FeatStruct(SBJ=sbj, OBJ=obj))
    return verb_fs.unify(syn_fs)

In [15]:
FeatStruct(SYN=FeatStruct(SBJ='Hund', OBJ='mir'))

[SYN=[OBJ='mir', SBJ='Hund']]

In [16]:
for sent in sentences:
    fs = semantic_parse(sent)
    print()
    print(sent)
    print(fs)


Der Hund ist mir bis nach Hause gefolgt!
[ CAT   = 'V'              ]
[ LEMMA = 'folgen'         ]
[                          ]
[ SEM   = [ AGT = 'Hund' ] ]
[         [ PAT = 'mir'  ] ]
[                          ]
[ SYN   = [ OBJ = 'mir'  ] ]
[         [ SBJ = 'Hund' ] ]

Das überrascht mich doch sehr!
[ CAT   = 'V'              ]
[ LEMMA = 'ueberraschen'   ]
[                          ]
[ SEM   = [ EXP = 'mich' ] ]
[         [ SRC = 'Das'  ] ]
[                          ]
[ SYN   = [ OBJ = 'mich' ] ]
[         [ SBJ = 'Das'  ] ]

Wer folgt wem auf Twitter?
[ CAT   = 'V'             ]
[ LEMMA = 'folgen'        ]
[                         ]
[ SEM   = [ AGT = 'Wer' ] ]
[         [ PAT = 'wem' ] ]
[                         ]
[ SYN   = [ OBJ = 'wem' ] ]
[         [ SBJ = 'Wer' ] ]

John überrascht Jane mit einer richtigen Antwort.
[ CAT   = 'V'              ]
[ LEMMA = 'ueberraschen'   ]
[                          ]
[ SEM   = [ EXP = 'Jane' ] ]
[         [ SRC = 'John' ] ]
[             

---
## Aufgabe 3: Adjektivdeklination

#### Die Adjektivdeklination im Deutschen bereitet vielen Deutschlernern Schwierigkeiten. Stellen Sie sich vor, Sie wollten einem Deutschlerner plausibel machen, dass die Adjektivdeklination keine reine Willkür ist.

#### Wählen Sie zur Illustration *das alte Bier* und *altes Bier*, und deklinieren Sie diese Nominalphrasen in allen vier Kasus.

#### Auf welche Gesetzmäßigkeiten würden Sie hinweisen? 

#### Beantworten Sie auch die folgenden Fragen!

| Nominativ | altes Bier | das alte Bier
| ---------- | ----------------- | ----------------- | 
| Genitiv | ... | ... | 
| Dativ | ... | ... | 
| Akkusativ | ... | ... | 

In [17]:
aufgabe(blatt6_3)

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie wird die Deklination der Nominalphrase <i>a…

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie wird die Deklination der Nominalphrase <i>d…

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Für die Deklination ist hier relevant, ob das A…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Aussagen über Adjektive stimmen?</h4>'…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Aussagen gelten für Nominalphrasen mit…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Aussagen gelten für den Fall, dass das…

---
## Aufgabe 4: Diathesen

#### Erläutern Sie, welche syntaktischen und welche kommunikativ-funktionalen Unterschiede zwischen den folgenden beiden Sätzen bestehen.

- *Hausmeister Meier ruft die Feuerwehr an.*
- *Die Feuerwehr wird von Hausmeister Meier angerufen.*

#### Beantworten Sie auch die folgenden Fragen.

In [18]:
aufgabe(blatt6_4)

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Schildern beide Sätze denselben Sachverhalt?</h…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche syntaktischen Unterschiede bestehen?</…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche kommunikativ-funktionalen Unterschiede…

---
## Aufgabe 5: Semantische Hierarchie

#### Vergleichen Sie die folgenden beiden Sätze:
- *Peter zerreißt Paul die Hose*
- *Die Hose zerreißt*

#### Wie kann im Rahmen einer funktionalen Syntaxtheorie (vgl. die semantische Hierarchie <code> Ag > Pat > Rec > Ben > Instr > Loc > Time </code>) erklärt werden, dass hier im Subjekt jeweils eine andere semantische Funktion/Rolle steht?

#### Beantworten Sie die folgenden Fragen.

In [19]:
aufgabe(blatt6_5)

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Was ist der bevorzugte Kandidat zur Realisierun…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Was passiert nach semantischer Hierarchie, we…

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Was ist im zweiten Satz das Subjekt?</h4>', lay…

---
## Aufgabe 6: Topik-Fokus

#### Konstruieren Sie einen Satz, in dem die fokussierte Information am Satzanfang steht. Stellen Sie diesen Satz in einen Minimalkontext, um nachzuweisen, dass das satzeinleitende Glied in der Tat im Fokus steht.

#### Ihr Satz könnte die Wörter *Affe* und *Tisch* beinhalten.

Wer sitzt auf dem Tisch? Ein Affe sitzt auf dem Tisch!

--- 
# Hausaufgaben

---
## Aufgabe 7: Kongruenz 2.0

#### Wie wir in Aufgabe 1 gesehen haben, besteht zwischen Artikel und Nomen einer Nominalphrase Kongruenz. Durch den Satzkontext haben wir erzwungen, dass Numerus und Kasus der Beispiele immer konstant waren (Singular, Nominativ). Lassen wir diese Einschränkung weg, ergeben sich noch weitere grammatische Kombinationsmöglichkeiten.

| - | Messer | Gabel | Löffel
| --- | ----------------- | ----------------- | ----------------- |
| der | Pl. Gen. | Sg. Gen./Dat. | Sg. Nom./Pl. Gen. |
| des | * | * | * |
| dem | Sg. Dat. | * | Sg. Dat. |
| den | * | * | Sg. Akk. |
| die | Pl. Nom./Akk. | Sg. Nom./Akk. | Pl. Nom./Akk. |
| das | Sg. Nom./Akk. | * | * |

#### Betrachten Sie das folgende mit Merkmalstrukturen implementierte Vollformlexikon. Schreiben Sie darauf aufbauend ein Programm, dass alle 18 Kombinationen aus obenstehender Tabelle als grammatisch oder ungrammatisch klassifiziert und im ersten Fall die unifizierte(n) Merkmalstruktur(en) ausgibt.

In [20]:
lexicon = {
    "Messer": [
        FeatStruct(GEN="neu", NUM="sg", KAS="nom"),
        FeatStruct(GEN="neu", NUM="sg", KAS="akk"),
        FeatStruct(GEN="neu", NUM="sg", KAS="dat"),
        FeatStruct(GEN="neu", NUM="pl", KAS="nom"),
        FeatStruct(GEN="neu", NUM="pl", KAS="akk"),
        FeatStruct(GEN="neu", NUM="pl", KAS="gen")
    ],
    "Gabel": [FeatStruct(NUM="sg", GEN="fem")],
    "Löffel": [
        FeatStruct(GEN="mas", NUM="sg", KAS="nom"),
        FeatStruct(GEN="mas", NUM="sg", KAS="akk"),
        FeatStruct(GEN="mas", NUM="sg", KAS="dat"),
        FeatStruct(GEN="mas", NUM="pl", KAS="nom"),
        FeatStruct(GEN="mas", NUM="pl", KAS="akk"),
        FeatStruct(GEN="mas", NUM="pl", KAS="gen"),
    ],

    "der": [
        FeatStruct(NUM="sg", GEN="mas", KAS="nom"),
        FeatStruct(NUM="sg", GEN="fem", KAS="dat"),
        FeatStruct(NUM="sg", GEN="fem", KAS="gen"),
        FeatStruct(NUM="pl", KAS="gen")
    ],
    "des": [
        FeatStruct(NUM="sg", GEN="mas", KAS="gen"),
        FeatStruct(NUM="sg", GEN="neu", KAS="gen")
    ],
    "dem": [
        FeatStruct(NUM="sg", GEN="mas", KAS="dat"),
        FeatStruct(NUM="sg", GEN="neu", KAS="dat"),
    ],
    "den": [
        FeatStruct(NUM="sg", GEN="mas", KAS="akk"),
        FeatStruct(NUM="pl", KAS="dat"),
    ],
    "die": [
        FeatStruct(NUM="sg", GEN="fem", KAS="nom"),
        FeatStruct(NUM="sg", GEN="fem", KAS="akk"),
        FeatStruct(NUM="pl", KAS="nom"),
        FeatStruct(NUM="pl", KAS="akk")
    ],
    "das": [
        FeatStruct(NUM="sg", GEN="neu", KAS="nom"),
        FeatStruct(NUM="sg", GEN="neu", KAS="akk"),
    ]
}

In [21]:
def predict(artikel, nomen):
    return True
    # == your code goes here == #

In [22]:
test_cases = [
    (det, noun)
    for det in ["der", "des", "dem", "den", "die", "das"]
    for noun in ["Messer", "Löffel", "Gabel"]
]
truth = [
    True, True, True,
    False, False, False,
    True, True, False,
    False, True, False,
    True, True, True,
    True, False, False
]

predictions = []
for artikel, nomen in test_cases:
    predictions.append(predict(artikel, nomen))

print(
    "Accuracy: {:.0f}%".format(
        accuracy_score(truth, predictions)*100
    )
)

Accuracy: 56%


---
## Aufgabe 8: Deklinationsmuster

#### Nach welchem Deklinationsmuster wurde in den folgenden Beispielen das Adjektiv dekliniert?

- *du blöder Kerl*
- *du dumme Gans*
- *du albernes Huhn*

#### Beantworten Sie auch die folgenden Fragen.

In [23]:
aufgabe(blatt6_8)

Lueckentext(children=(HTML(value='<i>Vervollständigen Sie den Lückentext: </i>', layout=Layout(justify_content…

---
## Aufgabe 9: Fragen zu NLTK-Kapitel 9.1

#### Lesen Sie das NLTK-Teilkapitel 9.1 (’Grammatical Features’).
#### Beantworten Sie insbesondere folgende Fragen:

---
### Aufgabe 9 a:

#### Worin besteht der Unterschied zwischen dem Einsatz von *Features* im POS-Tagging gegenüber dem Einsatz von *Features* in formalen Grammatiken?

In [24]:
aufgabe(blatt6_9a)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie werden Features beim POS-Tagging verwende…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie werden Features in formalen Grammatiken v…

---
### Aufgabe 9 b:

In [25]:
aufgabe(blatt6_9b)

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche native python-Datenstruktur bietet sich …

---
### Aufgabe 9 c:

In [26]:
aufgabe(blatt6_9c)

SelectionList(children=(HTML(value='<h4 style="font-size:14px;">Um welchen Faktor erhöht sich die Regelanzahl,…

---
## Aufgabe 10: Fragen zu NLTK-Kapitel 9.2

#### Lesen Sie das NLTK-Teilkapitel 9.2 (’Processing Feature Structures’): http://www.nltk.org/book/ch09.html.

#### Beantworten Sie insbesondere folgende Fragen:

---
### Aufgabe 10 a:
#### Welchen Wert hat der feature path <code>(’spouse’, ’address’, ’number’)</code> in (22):(http://www.nltk.org/book/ch09.html#ex-dag03) ? Geben Sie einen zu diesem Pfad äquivalenten Pfad an.

In [27]:
aufgabe(blatt6_10a)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Beantworten Sie die beiden obigen Fragen.</h4…

---
### Aufgabe 10 b:

#### Geben Sie zwei linguistische Merkmalstrukturen FS<sub>0</sub> und FS<sub>1</sub> an, die in einer Subsumptionsbeziehung stehen (FS<sub>0</sub> ⊑ FS<sub>1</sub> ). Was kommt nach (26), Kapitel 9.2.1, dabei heraus, wenn man solche zwei Merkmalstrukturen unifiziert?

In [28]:
aufgabe(blatt6_10b)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele erfüllen die obige Subsumpti…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Was gilt immer für das Ergebnis der Unifikati…

---
### Aufgabe 10 c:

In [29]:
aufgabe(blatt6_10c)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche der Beispiele erfüllen die Subsumption…

---
### Aufgabe 10 d:

In [30]:
aufgabe(blatt6_10d)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Im Falle, dass im Zuge einer Unifikation Info…

---
### Aufgabe 10 e:

In [31]:
aufgabe(blatt6_10e)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie läßt sich im NLTK ein äquivalenter Pfad d…